In [1]:
import gc
import os
import optuna
import joblib
import requests
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

from config import azuki_metadata, azuki_sales
from helper import add_rarity

metadata = azuki_metadata()
sales = azuki_sales()

pd.set_option('display.max_columns', None)

In [2]:
df = pd.DataFrame(list(metadata.find()))
df['tokenid'] = df['name'].apply(lambda x: int(x.split('#')[1]))
df.sort_values('tokenid', inplace=True)
df.index = df['tokenid']
df.drop(['_id', 'name', 'tokenid'], axis=1, inplace=True)
df.replace(np.nan, 'None', inplace=True)

metafields = df.columns[1:]
for column in metafields:
    col_ohe = pd.get_dummies(df[column].apply(pd.Series).stack(), prefix=f'{column}_meta').groupby(level=0).sum()
    df = pd.concat([df, col_ohe], axis=1).drop(column, axis=1)

rarity_added = add_rarity(df)

sales_hst = pd.DataFrame(list(sales.find()))
sales_hst['TOKENID'] = sales_hst['TOKENID'].astype('int')
sales_hst = sales_hst[['BLOCK_TIMESTAMP', 'PLATFORM_NAME', 'TOKENID', 'CURRENCY_SYMBOL', 'PRICE', 'PRICE_USD']].sort_values('BLOCK_TIMESTAMP')

final = pd.merge(rarity_added, sales_hst, left_index=True, right_on='TOKENID')
final.columns = [x.lower() for x in final.columns]
final.set_index('block_timestamp', inplace=True)

symbols = ['WETH', 'ETH']
final = final[final['currency_symbol'].isin(symbols)]


c:\Users\PC\Documents\Github\azuki_ml\project\helper.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[str(trait) + '_rarity'] = [( 1 / (data[trait].sum() / len(data))) if row == 1.0 else 0.0 for row in data[trait]]


In [6]:
final

image  \
BLOCK_TIMESTAMP                                                              
2022-07-10 02:21:37.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   
2022-07-10 03:09:41.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   
2022-02-01 06:49:51.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   
2022-02-10 19:59:53.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   
2022-05-10 03:06:22.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   
...                                                                    ...   
2022-08-12 12:59:49.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   
2022-01-26 07:20:46.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   
2022-01-22 05:12:46.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   
2022-05-10 19:49:03.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   
2022-05-19 16:12:18.000  https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...   

                         type_blue  type_human  type_red  type_spirit  \
BLOCK_TIMESTAMP                                                         
2022-07-10 02:21:37.000        0.0         1.0       0.0          0.0   
2022-07-10 03:09:41.000        0.0         1.0       0.0          0.0   
2022-02-01 06:49:51.000        0.0         1.0       0.0          0.0   
2022-02-10 19:59:53.000        0.0         1.0       0.0          0.0   
2022-05-10 03:06:22.000        0.0         1.0       0.0          0.0   
...                            ...         ...       ...          ...   
2022-08-12 12:59:49.000        0.0         1.0       0.0          0.0   
2022-01-26 07:20:46.000        0.0         1.0       0.0          0.0   
2022-01-22 05:12:46.000        0.0         1.0       0.0          0.0   
2022-05-10 19:49:03.000        0.0         1.0       0.0          0.0   
2022-05-19 16:12:18.000        0.0         1.0       0.0          0.0   

                         hair_black bangs  hair_black blonde half bun  \
BLOCK_TIMESTAMP                                                         
2022-07-10 02:21:37.000               0.0                         0.0   
2022-07-10 03:09:41.000               0.0                         0.0   
2022-02-01 06:49:51.000               0.0                         0.0   
2022-02-10 19:59:53.000               0.0                         0.0   
2022-05-10 03:06:22.000               0.0                         0.0   
...                                   ...                         ...   
2022-08-12 12:59:49.000               0.0                         0.0   
2022-01-26 07:20:46.000               0.0                         0.0   
2022-01-22 05:12:46.000               0.0                         0.0   
2022-05-10 19:49:03.000               0.0                         0.0   
2022-05-19 16:12:18.000               0.0                         0.0   

                         hair_black disheveled  hair_black fluffy  \
BLOCK_TIMESTAMP                                                     
2022-07-10 02:21:37.000                    0.0                0.0   
2022-07-10 03:09:41.000                    0.0                0.0   
2022-02-01 06:49:51.000                    0.0                0.0   
2022-02-10 19:59:53.000                    0.0                0.0   
2022-05-10 03:06:22.000                    0.0                0.0   
...                                        ...                ...   
2022-08-12 12:59:49.000                    0.0                0.0   
2022-01-26 07:20:46.000                    0.0                0.0   
2022-01-22 05:12:46.000                    0.0                0.0   
2022-05-10 19:49:03.000                    0.0                0.0   
2022-05-19 16:12:18.000                    0.0                0.0   

                         hair_black hairband  hair_black orange bangs  \
BLOCK_TIMESTAMP                                                         
2022-07-10 02:21:37.000                  0.0                      0.0   
2022-07-10 03:09:41.000              

In [ ]:
#sales.insert_many(azuki_sales.to_dict('records'))